In [1]:
pip install abjad


The following command must be run outside of the IPython shell:

    $ pip install abjad

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [1]:
import ipywidgets as widgets
from IPython.display import clear_output
from random import randrange
import abjad, dca_functions

In [2]:
# default declaration, can change with slider/button
pitches = list(range(0, 12))

pitch_range_slider = widgets.IntRangeSlider(
    value=[0, 11],
    min=0,
    max=11, # make this larger to include more than one octave
    description='Pitch Range:', # chromatic pitches
)

display(pitch_range_slider)

# can change this to offset from middle C
offset = 0

# Number of pitches in sequence
length = widgets.IntSlider(
    value=50,
    min=1,
    max=500,
    step=1,
    description='Length (# notes):',
    style={'description_width': 'initial', 'width': '800px'},
    readout=True,
)
display(length)
    


IntRangeSlider(value=(0, 11), description='Pitch Range:', max=11)

IntSlider(value=50, description='Length (# notes):', max=500, min=1, style=SliderStyle(description_width='init…

In [3]:
random_PS_button = widgets.Button(
    description='Make random pitch sequence',
    layout=widgets.Layout(width='200px'),
)
output = widgets.Output()

display(random_PS_button, output)

def on_button_clicked(b):
    with output:
        clear_output()
        print(dca_functions.make_random_pitch_sequence(offset, length, pitch_range_slider))

random_PS_button.on_click(on_button_clicked)

Button(description='Make random pitch sequence', layout=Layout(width='200px'), style=ButtonStyle())

Output()

# Notice the repetitions

In [4]:
dca_PS_button = widgets.Button(
    description='Make dissonant counterpoint spaced pitch sequence',
    layout=widgets.Layout(width='350px'),
)

display(pitch_range_slider)
display(length)


weights_option = widgets.Dropdown(
    options=['equal', 'favor lower', 'favor center', 'favor upper'],
    value='equal',
    description='Weight options:',
    style={'description_width': 'initial', 'width': '800px'},
)

print_option = widgets.ToggleButtons(
    options=["Don't Print", "Print Notation"],
    description='Print Option:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

display(weights_option)

display(dca_PS_button, output)

def on_dca_button_clicked(b):
    with output:
        clear_output()
        sequence = dca_functions.make_dca_spaced_pitch_sequence(offset, weights_option, length, pitch_range_slider, show_data=True)
        print("Sequence:", sequence)
        
        pitches = list(range(pitch_range_slider.value[0], pitch_range_slider.value[1]+1))
        print("\nPitch Frequency (total times each pitch occurs in sequence:")
        for pitch in pitches:
            print(pitch, ":", sequence.count(pitch))
        
dca_PS_button.on_click(on_dca_button_clicked)

IntRangeSlider(value=(0, 11), description='Pitch Range:', max=11)

IntSlider(value=50, description='Length (# notes):', max=500, min=1, style=SliderStyle(description_width='init…

Dropdown(description='Weight options:', options=('equal', 'favor lower', 'favor center', 'favor upper'), style…

Button(description='Make dissonant counterpoint spaced pitch sequence', layout=Layout(width='350px'), style=Bu…

Output()

# After an element is chosen, it's count goes to 0 and it's probability goes to near-zero. Higher counts are biased by the growth function (2**n) to have even higher probabilities.

# See any repetitions? Probably not!

# Here's what the pitch frequency looks like:

# what if we change the weights to favor certain pitches?

# Still some randomness, but definitely skewed towards one side

# Let's reset the weights to an equal value and try that again

# Now let's see what this looks like in notation

In [5]:
display(print_option)


dca_PS_button2 = widgets.Button(
    description='Make dissonant counterpoint spaced pitch sequence',
    layout=widgets.Layout(width='350px'),
)

display(pitch_range_slider)
display(length)
display(weights_option)
display(dca_PS_button2, output)

def on_dca_button2_clicked(b):
    with output:
        clear_output()
        sequence = dca_functions.make_dca_spaced_pitch_sequence(offset, weights_option, length, pitch_range_slider, show_data=False, print_ly=True)
        print("Sequence:", sequence)
        
        pitches = list(range(pitch_range_slider.value[0], pitch_range_slider.value[1]+1))
        print("\nPitch Frequency (total times each pitch occurs in sequence:")
        for pitch in pitches:
            print(pitch, ":", sequence.count(pitch))
        
dca_PS_button2.on_click(on_dca_button2_clicked)



ToggleButtons(description='Print Option:', options=("Don't Print", 'Print Notation'), value="Don't Print")

IntRangeSlider(value=(0, 11), description='Pitch Range:', max=11)

IntSlider(value=50, description='Length (# notes):', max=500, min=1, style=SliderStyle(description_width='init…

Dropdown(description='Weight options:', options=('equal', 'favor lower', 'favor center', 'favor upper'), style…

Button(description='Make dissonant counterpoint spaced pitch sequence', layout=Layout(width='350px'), style=Bu…

Output()

In [ ]:
sequence = make_dca_spaced_pitch_sequence(pitches, offset, weights, length)
output_ly(sequence)

# How about changing over time?

1. make weights a function of their distance from a pitch center
2. move that pitch center over time
3. make sequences (clangs) vary in length
4. increase pitch center and sequence average length until 2/3 through form, then decrease

In [ ]:
def make_clang(pitches, weights, length, deviation, offset=0):
    # length = average length +/- deviation range
    length = length + randrange(deviation*-1, deviation)
    sequence = make_dca_spaced_pitch_sequence(pitches, offset, weights, length)
    sequence.append("rest") # end every clang with a rest
    return sequence

In [ ]:
# build clangs
total_clangs = 72

# weight by proximity to pitch center
increasing = [ 1 * i for i in range(1, 7) ]
decreasing = list(reversed(increasing))
weights = increasing + decreasing

# make peak 2/3 of the way through form. Tenney does this with a calculation based on the golden mean, but is more/less in the same place
peak = int(total_clangs * (2/3))

length = 5
length_deviation = int(length * 0.5)

offset = -12
pitch_center = 6
clangs = []

In [ ]:
for clang in range(total_clangs):
    pitches = [*range(int(pitch_center) - 6, int(pitch_center+6))]
    #print(pitches)

    sequence = make_clang(pitches, weights, length, length_deviation, offset=offset)
    clangs.append(sequence)

    if clang < peak:
        pitch_center += 0.6
        length += 1
    else:
        pitch_center -= 1.25
        length -= 2
    length_deviation = int(length * 0.5)

In [ ]:
for clang in clangs:
    print(clang)

In [ ]:
def output_clangs(clangs):
    notes = []
    for clang in clangs:
        for pitch in clang:
            if pitch == "rest":
                notes.append(abjad.Rest('r8'))
            else:
                duration = abjad.Duration(1, 8)
                note = abjad.Note(pitch, duration)
                notes.append(note)

    staff = abjad.Staff(notes)
    abjad.show(staff)
    abjad.play(staff)

In [ ]:
output_clangs(clangs)